# import some libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer as vec
from sklearn.svm import LinearSVC
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_val_score
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
wnl = WordNetLemmatizer()
 
print("rocks :", wnl.lemmatize("rocks", pos = 'n'))
print("corpora :", wnl.lemmatize("corpora", pos = 'n'))


rocks : rock
corpora : corpus


# Take datasets from CSV files

In [61]:
# df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [47]:
# Open the file in read mode
reviews_content = []
sentiment = []
with open('cheat.csv', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Process each line as needed
        line = line.strip()
        reviews_content.append(line[2::])
        sentiment.append(line[0])   

In [48]:
print(len(sentiment))

2000


In [49]:
print(len(reviews_content))

2000


In [52]:
df_train = pd.DataFrame({'reviews_content':reviews_content,'category':sentiment})
df_train['category'] = df_train['category'].map({'0': 'negative', '1': 'positive'})

In [55]:
df_train.to_csv('Github.csv',index=False)

In [54]:
df_train

,reviews_content,category
0,"originally launched in 1978 , this popular fil...",positive
1,"confucius once said , "" governing a nation is ...",negative
2,ever wonder what happened to gabe kaplan ? yo...,positive
3,"for this review and more , visit clear illusio...",positive
4,"errol morris , critically acclaimed director o...",positive
...,...,...
1995,as forgetful as some people may be it is doubt...,negative
1996,""" through a spyglass , i could see everything...",positive
1997,sometimes i wonder just what the censors are t...,negative
1998,a frequent error is the categorization of a te...,positive


# define function for reprocessing

In [56]:
def extract_alphabetic(text):
    # Use regular expression to remove non-alphabetic characters
    return re.sub(r'[^a-zA-Z\s]', '', text)
def wordnet_lemmatizer(text):
    words = text.split()
    output = [wnl.lemmatize(element) for element in words]
    return ' '.join(output)

In [63]:
df_train['processed text'] = df_train['reviews_content'].apply(extract_alphabetic)
df_train['processed text'] = df_train['processed text'].apply(wordnet_lemmatizer)

df_test['processed text'] = df_test['reviews_content'].apply(extract_alphabetic)
df_test['processed text'] = df_test['processed text'].apply(wordnet_lemmatizer)

# Perform TFIDF

In [59]:
vect = vec(ngram_range = (1,4),min_df = 8,max_df = 1000, norm= 'l2')
vect.fit(df_train['processed text'])
X_train = vect.transform(df_train['processed text'])
y_train = df_train['category']


In [64]:
X_test = vect.transform(df_test['processed text'])

# fine-tuning regularization constant C

In [60]:
for i in np.arange(0.1,2,0.1):
    model = LinearSVC(tol = 0.001, C = i,dual ='auto')
    result = cross_val_score(model, X_train,y_train,cv = 15)
    print(f'C = {i:.2f} avg - {np.mean(result):.2f} median - {np.median(result):.2f}')

C = 0.10 avg - 0.86 median - 0.87
C = 0.20 avg - 0.87 median - 0.87
C = 0.30 avg - 0.87 median - 0.87
C = 0.40 avg - 0.87 median - 0.89
C = 0.50 avg - 0.88 median - 0.89
C = 0.60 avg - 0.88 median - 0.89
C = 0.70 avg - 0.88 median - 0.89
C = 0.80 avg - 0.88 median - 0.89
C = 0.90 avg - 0.88 median - 0.89
C = 1.00 avg - 0.88 median - 0.88
C = 1.10 avg - 0.88 median - 0.88
C = 1.20 avg - 0.88 median - 0.88
C = 1.30 avg - 0.88 median - 0.88
C = 1.40 avg - 0.88 median - 0.88
C = 1.50 avg - 0.88 median - 0.87
C = 1.60 avg - 0.88 median - 0.87
C = 1.70 avg - 0.88 median - 0.87
C = 1.80 avg - 0.88 median - 0.87
C = 1.90 avg - 0.88 median - 0.88


now we choose $C = 0.9$

# Train the model

In [69]:
model = LinearSVC(tol = 0.001, C = 0.9, dual = 'auto')
model.fit(X_train,y_train)
# y_predict = model.predict(X_test)
y_predict = model.predict(X_test)
# accuracy_score(y_predict,y_train)

# Make the CSV files to upload

In [71]:
output_file = pd.DataFrame({'Row': range(1,501), 'Label':y_predict})
output_file.to_csv('SVM_Jokes.csv',index = False)

In [79]:
df_curang =  pd.read_csv('SVM_Jokes.csv')
df_curang['Label'].value_counts()

Label
negative    252
positive    248
Name: count, dtype: int64